# 2 SpilloverMeasures

In [3]:
%load_ext autoreload
%autoreload 2
from lib.base import *

In [4]:
from tqdm.notebook import tqdm
import numpy as np
from statsmodels.tsa.api import VAR
from lib.gfevd import compute_gfevd
import pandas as pd
from lib.stat import adf_check


# 读取数据

In [1]:
from lib.data import load_vol, load_price
open_df, high_df, low_df, close_df = load_price()

In [5]:
from lib.data import get_hz_esg_data
df_hz_esg = get_hz_esg_data(time_type='season')
df_hz_esg['评级日期'] = pd.to_datetime(df_hz_esg['评级日期'])

/Users/xhc/Documents/Research/[文章]ESG volatility China/[Github]Volatility-Spillovers-ESG-China
/Users/xhc/Documents/Research/[文章]ESG volatility China/[Github]Volatility-Spillovers-ESG-China/datasets/华证2009-2023年（含细分项+季度)）/华证esg评级2009-2023（细分项）/华证esg评级含细分项（季度）2009-2023.xlsx


## 计算波动率

In [6]:
from lib.volatility import calculate_volatility
vol_interday = calculate_volatility(open_df, high_df, low_df, close_df)

## 稳定性检验
剔除不稳定的股票

In [ ]:
from lib.stat import adf_check
vol_df = adf_check(vol_df)

## VAR和方差分解

In [69]:
season = df_hz_esg['评级日期'].unique()
steps = 10
for i in tqdm(season):
    i = pd.to_datetime(i)
    # 按季度计算波动率
    df_hz_esg_i = df_hz_esg[df_hz_esg['评级日期'] == i]
    code_list_i = np.intersect1d(df_hz_esg_i.index.values, vol_interday.columns.values)
    logic = (vol_interday.index.month == i.month) * (vol_interday.index.year == i.year)
    if logic.any() == True: 
        print(i)
    else:
        continue
    vol_interday_i = vol_interday[logic]
    code_list_i = np.intersect1d(vol_interday.columns.values, code_list_i)
    volatilities_i = vol_interday_i[code_list_i]
    volatilities_i = volatilities_i.loc[:, (volatilities_i.isnull().sum()>200) == False].fillna(method = 'ffill').fillna(method ='pad')
    volatilities_i = volatilities_i.dropna(axis = 1)

    print("vol shape before ADF", volatilities_i.shape)
    # 标准化
    volatilities_i = (volatilities_i - volatilities_i.mean()) / volatilities_i.std()
    volatilities_i = volatilities_i.dropna()
    # 全部都是一样的数据删除
    constant_columns = volatilities_i.columns[volatilities_i.nunique() < 10]
    # 删除这些列
    volatilities_i = volatilities_i.drop(columns=constant_columns)
    #P平稳性检验
    #volatilities_i = adf_check(volatilities_i)
    print("vol shape after ADF", volatilities_i.shape)

    #计算方差分解的矩阵  这里是全网络的生成
    model = VAR(volatilities_i)
    p = 1  # 1,2,3,4
    results = model.fit(p) # p: 滞后阶数
    # 获取模型残差的协方差矩阵及其逆矩阵    
    # resid = results.resid
    # cov_matrix = np.cov(resid, rowvar=False)

    sigma_u = results.sigma_u.values
    A = results.ma_rep(steps)
    H = 10   # 预测误差的步数 2,5,10,20
    gfevd = compute_gfevd(H, A, sigma_u)
    gfevd_df = pd.DataFrame(gfevd, index=volatilities_i.columns, columns=volatilities_i.columns)

    # 按行做归一化
    #gfevd_df_nor = normalize_rows_with_non_negative(gfevd_df)

    gfevd_df.to_csv('outputs/season/' + str(i)[:10] + '.csv')

  0%|          | 0/60 [00:00<?, ?it/s]

2010-01-31 00:00:00
vol shape before ADF (20, 1523)
vol shape after ADF (20, 1521)
2010-04-30 00:00:00
vol shape before ADF (21, 1614)
vol shape after ADF (21, 1608)
2010-07-31 00:00:00
vol shape before ADF (22, 1691)
vol shape after ADF (0, 0)


ValueError: zero-size array to reduction operation maximum which has no identity